In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os

notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break
 
if commit_fit_path is None:
    raise ValueError("Path containing 'ensemble_commit' not found in notebook path.")


if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

# import ensemble_model.preprocesser as preprocesser 
# import ensemble_model.MoE_model_CE_loss as moe 
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 

from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer, DistilBertModel


root
ensemble_commit
/root/ensemble_commit


In [2]:
df = pd.read_json(r'../datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df

train, test = train_test_split(df, test_size=0.3, random_state=42)
test, val = train_test_split(test, test_size=0.5, random_state=42)
train,_ = train_test_split(train, train_size=0.1, random_state=42)
test, val = train_test_split(test, train_size=0.1, random_state=42)
val, _ = train_test_split(val, train_size=0.1, random_state=42)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

/tmp/ipykernel_1775/2474789719.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


In [3]:
# df['command'] = df['diff'].apply(lambda x : extract_lines(find_longest_list([y.changes for y in whatthepatch.parse_patch(x)])))
# extracted_cols = df['diff'].str.extract(r'^diff --git (?P<source>[^\t\n]+) (?P<target>[^\t\n]+)')
# whatthepatch.apply_diff(diff, lao)

In [4]:
# len(df_dataset)

In [5]:
# df_dataset[0]

In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('../models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

codebert_model = RobertaModel.from_pretrained('../models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('../models/codebert-base')

Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer, message='commit_message',command='diff_code',label='category')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer, message='commit_message',command='diff_code',label='category')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer, message='commit_message',command='diff_code',label='category')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

In [8]:
# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 0.4697: 100%|██████████| 314/314 [01:54<00:00,  2.73batch/s]


=============================train========================


Epoch 2/10 Loss: 0.2623: 100%|██████████| 314/314 [01:49<00:00,  2.86batch/s]


=============================train========================


Epoch 3/10 Loss: 0.1510: 100%|██████████| 314/314 [01:48<00:00,  2.90batch/s]


...

=============================train========================


Epoch 9/10 Loss: 0.0232: 100%|██████████| 314/314 [01:48<00:00,  2.90batch/s]


=============================train========================


Epoch 10/10 Loss: 0.0131: 100%|██████████| 314/314 [01:48<00:00,  2.91batch/s]


=============================train========================
Early stopping triggered after 10 epochs
Validation Accuracy: 0.8778
Precision: 0.8846
Recall: 0.8778
F1-Score: 0.8726
              precision    recall  f1-score   support

           0     0.8587    0.9748    0.9131       318
           1     0.9344    0.6909    0.7944       165

    accuracy                         0.8778       483
   macro avg     0.8966    0.8329    0.8538       483
weighted avg     0.8846    0.8778    0.8726       483



In [9]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.9032
Precision: 0.9100
Recall: 0.9032
F1-Score: 0.8985
              precision    recall  f1-score   support

           0     0.8841    0.9892    0.9337       370
           1     0.9675    0.7126    0.8207       167

    accuracy                         0.9032       537
   macro avg     0.9258    0.8509    0.8772       537
weighted avg     0.9100    0.9032    0.8985       537

